# Remotes

Consider features of working with remotes in git. Check corresponding section in [git documentation](https://git-scm.com/book/ms/v2/Git-Basics-Working-with-Remotes).

## Set up

Obviously, we need remotes for this section. To avoid repetitive manual operations like creating and deleting repositories in Git storage providers, we'll need a specific setup.

The following cell shows a Dockerfile that creates an image for containers acting as toy servers. Learn more about setting up a Git daemon in the [special section](https://git-scm.com/book/en/v2/Git-on-the-Server-Git-Daemon) of the official documentation.

In [1]:
cat remote_files/dockerfile

FROM debian:bullseye-slim

# Install git (including git-daemon)
RUN apt-get update && apt-get install -y git && apt-get clean

# Create a directory for repositories and initialize a bare repository
RUN mkdir -p /git/project.git && git init --bare /git/project.git

# Add a file to allow git-daemon to serve the repository
RUN touch /git/project.git/git-daemon-export-ok

# Expose the Git protocol port
EXPOSE 9418

# Start git-daemon to serve repositories
CMD ["git", "daemon", "--verbose", "--export-all", "--base-path=/git", "/git"]


This dockerfile installs all the required packages on Debian Linux and creates an empty git repository, which we will basically use as an example.

Build image:

In [5]:
docker build -f remote_files/dockerfile -t git-daemon . &> /dev/null

Running container:

In [5]:
docker run -itd --rm --name git-daemon -p 9418:9418 git-daemon

f004a8ef4fdb696621e0c6d81ad5a4ba97aeb8379942ec8bef505e4267d3d916


Now you can clone the repository into a local folder. The following code performs the cloning and displays the result of a Git commit from the repository.

In [7]:
git clone git://localhost:9418/project.git /tmp/remote
cd /tmp/remote
git status

Cloning into '/tmp/remote'...
On branch master

No commits yet

nothing to commit (create/copy files and use "git add" to track)


**Note:** Don't forget to clean up the environment when you're done.

In [ ]:
docker stop git-daemon

git-daemon


## Show remotes

You can use the `git remote` command to list the short names of remotes you're dealing with. With `git remove -v` you can show specific urls.

---

The following cell shows the result of the `git remote` command.

In [8]:
git remote

origin


`origin` is default name of the remote gitven by `git clone`.

The following command shows the result of the `git remove -v` command.

In [9]:
git remote -v

origin	git://localhost:9418/project.git (fetch)
origin	git://localhost:9418/project.git (push)


Now for remote consideration specified fetch and pull URLs.